In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# import clip
import torch
torch.set_num_threads(6)


In [ ]:
from tools import (DDGSearch,LinuxShell,PythonInterpreter,WikipediaSummary)
from utils.docker import DockerInterface


In [35]:
prompt = """
Today is {today} and you can use tools to get new information.
Answer the question as best as you can using the following tools:

{tool_description}


Use the following format:

Question: the input question you must answer
Thought: comment on what you want to do next
Action: the action to take, exactly one element of [{tool_names}]
Action Input: the input to the action  (Never predict the action output.)
 
Begin!

Question: {question}
Thought: {previous_responses}"""

In [7]:
available_tools = {
    "DuckDuckGo Search": DDGSearch,
    "Linux Shell": LinuxShell,
    "Python Interpreter": PythonInterpreter,
    
    "Wikipedia": WikipediaSummary,
}

tools_config = {
            "DuckDuckGo Search": {
                "max_results": 3,
                "instant_answers": False,
                "safe_search": True,
                "time_limit": True,
                "add_url": False,
                "confirm_action": False,
            },
            "Linux Shell": {
                "container_name": "linux_repl",
                "image": "alpine:3.14",
                "image_name": "agentic_llm_linux",
                "persistent_container": True,
                "confirm_action": True,
            },
            "Python Interpreter": {
                "container_name": "python_repl",
                "image": "python:3.9-slim-bookworm",
                "image_name": "agentic_llm_python",
                "persistent_container": True,
                "confirm_action": True,
            },
            "Wikipedia": {
                "max_results": 1,
                "max_words": 200,
                "language": "en",
                "confirm_action": False}}

In [8]:
tools = {
    tool_name: available_tools[tool_name](**tool_cfg)
    for tool_name, tool_cfg in tools_config.items()
}

_tool_usage = "\n".join(
    [f"{tool.name}: {tool.usage}" for tool in tools.values()]
)

_tool_names = ", ".join(list(tools.keys()))

starting container linux_repl...
...done!
starting container python_repl...
...done!


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto", 
)
tokenizer = AutoTokenizer.from_pretrained(model_name)



/home/sonhamin/anaconda3/envs/codellm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:22<00:00,  5.57s/it]


In [10]:
# question = "Use Python to calculate: what is 7*7+4?"
question = "How many 1s are in 1111111111111111111111111?"

In [11]:
len('1111111111111111111111111')

25

# No tool usage  - answer is incorrect

In [65]:
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": question}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)


response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
How many 1s are in 1111111111111111111111111?
assistant
The number 1111111111111111111111111 is composed of 32 ones.


# However, if we ask it to write a python script to count the number of ones, it does it correctly

In [69]:
question = "Write a python script to count how many 1s there are in 1111111111111111111111111."

In [70]:
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": question}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)


response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Write a python script to count how many 1s there are in 1111111111111111111111111.
assistant
Certainly! Here is a simple Python script that counts the number of `1`s in the given number:

```python
# Define the number as a string for easy counting
number = "1111111111111111111111111"

# Count the number of '1's
count_of_ones = number.count('1')

# Print the result
print(f"The number of 1s in {number} is: {count_of_ones}")
```

When you run this script, it will output:

```
The number of 1s in 1111111111111111111111111 is: 32
```

This script works by converting the number into a string and then using the `count` method to determine how many times the character `'1'` appears in the string.


# Code written by the LLM

In [71]:

# Define the number as a string for easy counting
number = "1111111111111111111111111"

# Count the number of '1's
count_of_ones = number.count('1')

# Print the result
print(f"The number of 1s in {number} is: {count_of_ones}")

The number of 1s in 1111111111111111111111111 is: 25


# Tool usage

In [78]:
import datetime

question = "How many 1s are in 1111111111111111111111111?"

system_prompt = prompt.format(
    today=datetime.date.today(),
    tool_description=_tool_usage,
    tool_names=_tool_names,
    question=question,
    previous_responses="{previous_responses}",
)  

In [79]:
print(system_prompt)


Today is 2025-04-16 and you can use tools to get new information.
Answer the question as best as you can using the following tools:

DDG Search: Get information for a search query. Input should be either a question like 'Who was Tolstoy?', or a single word like 'Tolstoy'. Result will be the answers to the question. There can be multiple results. 
Linux Shell: A Linux Shell. Use this to execute Linux commands. Input should be valid Linux commands. Chain several commands together with semicolon. Missing packages can be installed with `apk add ...` 
Python Interpreter: A Python interpreter. Use this to execute python code. Input should be valid python code. If you want to see the output of a value, you should print it out with `print(...)`
Wikipedia: Get information from an encyclopedia to answer a search query. Input should be a search query such as 'Who was Marie Curie?'. Result will be a detailed answer to the question. There can be multiple results. 


Use the following format:

Ques

In [80]:
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": system_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)


In [81]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

In [82]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


# The codeLLM wants to use python and provides the code

In [83]:
print(response)

Question: How many 1s are in 1111111111111111111111111?
Thought: To count the number of 1s in the given number, I can convert it to a string and then count the occurrences of the digit '1'.
Action: Python Interpreter
Action Input:
```python
number = "1111111111111111111111111"
count_of_ones = number.count('1')
print(count_of_ones)
```


# The result of the code is '25\n' - we pass this back to the LLM

In [85]:
import typing
import re

def _parse_response(response: str) -> typing.Tuple[str, str]:
    if "Final Answer:" in response:
        return "Final Answer:", response.split("Final Answer:")[-1].strip()
    match = re.search(
        r"Action: [\[]?(.*?)[\]]?[\n]*Action Input:[\s]*(.*)",
        response,
        re.DOTALL,
    )
    if not match:
        raise ValueError(f"Can't interpret LLM's response: {response}")
    action = match.group(1).strip()
    action_value = match.group(2)
    return action, action_value.strip(" ").strip('"').replace("`", "")

In [86]:
action, action_value = _parse_response(response)
action_value = action_value.replace("python\n", "")

In [87]:
result = tools[action](action_value)

In [88]:
result

'25\n'

In [90]:
response += f"\nObservation: {result}\nThought:"
print(response)

this_prompt = system_prompt.format(
    previous_responses=response
)

Question: How many 1s are in 1111111111111111111111111?
Thought: To count the number of 1s in the given number, I can convert it to a string and then count the occurrences of the digit '1'.
Action: Python Interpreter
Action Input:
```python
number = "1111111111111111111111111"
count_of_ones = number.count('1')
print(count_of_ones)
```
Observation: 25

Thought:


# It provides the final answer - 25

In [92]:
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": this_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user

Today is 2025-04-16 and you can use tools to get new information.
Answer the question as best as you can using the following tools:

DDG Search: Get information for a search query. Input should be either a question like 'Who was Tolstoy?', or a single word like 'Tolstoy'. Result will be the answers to the question. There can be multiple results. 
Linux Shell: A Linux Shell. Use this to execute Linux commands. Input should be valid Linux commands. Chain several commands together with semicolon. Missing packages can be installed with `apk add ...` 
Python Interpreter: A Python interpreter. Use this to execute python code. Input should be valid python code. If you want to see the output of a value, you should print it out with `print(...)`
Wikipedia: Get information from an encyclopedia to answer a search query. Input should be a search query such as 'Who was Marie Curie?'. Result will be a detailed answer t

# Other tools already implemented

In [98]:
print(tools['Wikipedia']('News'))

title: News
summary: News is information about current events. This may be provided through many different media: word of mouth, printing, postal systems, broadcasting, electronic communication, or through the testimony o
source: https://en.wikipedia.org/wiki/News


In [97]:
print(tools['DuckDuckGo Search']('News'))

title: News: U.S. and World News Headlines : NPR
answer: NPR news, audio, and podcasts. Coverage of breaking stories, national and world news, politics, business, science, technology, and extended coverage of major national and world events.
title: The New York Times - Breaking News, US News, World News and Videos
answer: Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news ...
title: Shooting at Davis community park leaves 3 injured | abc10.com
answer: The UC Davis community is saddened by the news that individuals enjoying a day at Community Park in Davis were injured in a shooting Saturday afternoon. That the incident occurred as thousands of ...


In [96]:
print(tools['Linux Shell']('ls -al'))

total 8
drwxr-xr-x    2 root     root          4096 Apr 16 21:55 .
drwxr-xr-x    1 root     root          4096 Apr 16 21:55 ..



In [ ]:
# prompt = """
# Today is {today} and you can use tools to get new information.
# Answer the question as best as you can using the following tools:

# {tool_description}


# Use the following format:

# Question: the input question you must answer
# Thought: comment on what you want to do next
# Action: the action to take, exactly one element of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation repeats N times,
#  use it until you are sure of the answer)
# Thought: I now know the final answer
# Final Answer: your final answer to the original input question

# Begin!

# Question: {question}
# Thought: {previous_responses}"""
# def run_python_code(code_to_run):
#     output = !python -c "{code_to_run}"
#     return output
    